# Analyze the sales of the company in order to target new areas of growth

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statistics
from scipy import stats
from scipy.stats import chi2_contingency
from datetime import datetime

In [2]:
# Load Transaction Data
transactions = pd.read_csv('transactions.csv')
print(transactions)

       id_prod                        date session_id client_id
0       0_1483  2021-04-10 18:37:28.723910    s_18746    c_4450
1        2_226  2022-02-03 01:55:53.276402   s_159142     c_277
2        1_374  2021-09-23 15:13:46.938559    s_94290    c_4270
3       0_2186  2021-10-17 03:27:18.783634   s_105936    c_4597
4       0_1351  2021-07-17 20:34:25.800563    s_63642    c_1242
...        ...                         ...        ...       ...
337011   1_671  2021-05-28 12:35:46.214839    s_40720    c_3454
337012   0_759  2021-06-19 00:19:23.917703    s_50568    c_6268
337013  0_1256  2021-03-16 17:31:59.442007     s_7219    c_4137
337014   2_227  2021-10-30 16:50:15.997750   s_112349       c_5
337015  0_1417  2021-06-26 14:38:19.732946    s_54117    c_6714

[337016 rows x 4 columns]


In [3]:
# check to clean for bad data
transactions.describe()

,id_prod,date,session_id,client_id
count,337016,337016,337016,337016
unique,3266,336855,169195,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,1081,13,200,12855


In [4]:
# Replace bad data (date starting with test)
mask = transactions['date'].str.startswith('test_')
transactions.loc[mask, 'date'] = transactions.loc[mask, 'date'].str.replace('test_', '')
print(mask)

0         False
1         False
2         False
3         False
4         False
          ...  
337011    False
337012    False
337013    False
337014    False
337015    False
Name: date, Length: 337016, dtype: bool


In [5]:
# check again to clean for bad data
transactions.describe()

,id_prod,date,session_id,client_id
count,337016,337016,337016,337016
unique,3266,336855,169195,8602
top,1_369,2021-03-01 02:30:02.237413,s_0,c_1609
freq,1081,13,200,12855


In [6]:
# Check for missing values in data
transactions.isnull().sum().sum()

0

In [7]:
# Check missing by variable
transactions.isnull().sum()

id_prod       0
date          0
session_id    0
client_id     0
dtype: int64

In [8]:
# load Product Data
products = pd.read_csv('products.csv')
print(products)

     id_prod   price  categ
0     0_1421   19.99      0
1     0_1368    5.13      0
2      0_731   17.99      0
3      1_587    4.99      1
4     0_1507    3.99      0
...      ...     ...    ...
3282    2_23  115.99      2
3283   0_146   17.14      0
3284   0_802   11.22      0
3285   1_140   38.56      1
3286  0_1920   25.16      0

[3287 rows x 3 columns]


In [9]:
# check to clean for bad data
products.describe()

,price,categ
count,3287.000000,3287.000000
mean,21.856641,0.370246
std,29.847908,0.615387
min,-1.000000,0.000000
25%,6.990000,0.000000
50%,13.060000,0.000000
75%,22.990000,1.000000
max,300.000000,2.000000


In [10]:
# Remove negative 'price' values
cleaned_products = products[products['price'] >= 0]
print(cleaned_products)

     id_prod   price  categ
0     0_1421   19.99      0
1     0_1368    5.13      0
2      0_731   17.99      0
3      1_587    4.99      1
4     0_1507    3.99      0
...      ...     ...    ...
3282    2_23  115.99      2
3283   0_146   17.14      0
3284   0_802   11.22      0
3285   1_140   38.56      1
3286  0_1920   25.16      0

[3286 rows x 3 columns]


In [11]:
# check to clean for bad data
cleaned_products.describe()

,price,categ
count,3286.000000,3286.000000
mean,21.863597,0.370359
std,29.849786,0.615446
min,0.620000,0.000000
25%,6.990000,0.000000
50%,13.075000,0.000000
75%,22.990000,1.000000
max,300.000000,2.000000


In [12]:
# Check missing by variable
products.isnull().sum()

id_prod    0
price      0
categ      0
dtype: int64

In [13]:
# Perform regular/inner join instead to only bring data that exists in both tables
d1 = transactions.merge(cleaned_products, left_on = 'id_prod', right_on ='id_prod')
print(d1)

       id_prod                        date session_id client_id   price  categ
0       0_1483  2021-04-10 18:37:28.723910    s_18746    c_4450    4.99      0
1       0_1483  2021-10-18 19:16:14.767807   s_106741    c_1576    4.99      0
2       0_1483  2021-11-14 11:11:45.276771   s_119491     c_182    4.99      0
3       0_1483  2021-07-30 19:37:16.087057    s_69222    c_1196    4.99      0
4       0_1483  2021-03-30 14:18:13.403218    s_13545    c_2890    4.99      0
...        ...                         ...        ...       ...     ...    ...
336708  0_1683  2021-12-24 04:37:46.407713   s_139091    c_5508    2.99      0
336709    2_98  2021-03-08 21:10:32.250919     s_3637    c_5967  149.74      2
336710   0_833  2021-09-27 23:22:40.394509    s_96558    c_8260    2.99      0
336711  0_1284  2021-04-14 13:13:06.560397    s_20477    c_1551    1.38      0
336712  0_1116  2021-07-27 04:02:53.408409    s_67633    c_3291    2.97      0

[336713 rows x 6 columns]


In [14]:
# check to clean for bad data
d1.describe()

,price,categ
count,336713.000000,336713.000000
mean,17.215189,0.430156
std,17.855445,0.591082
min,0.620000,0.000000
25%,8.610000,0.000000
50%,13.900000,0.000000
75%,18.990000,1.000000
max,300.000000,2.000000


In [15]:
# Check missing by variable
d1.isnull().sum()

id_prod       0
date          0
session_id    0
client_id     0
price         0
categ         0
dtype: int64

In [16]:
# Load Customer Data
customers = pd.read_csv('customers.csv')
print(customers)

     client_id sex  birth
0       c_4410   f   1967
1       c_7839   f   1975
2       c_1699   f   1984
3       c_5961   f   1962
4       c_5320   m   1943
...        ...  ..    ...
8618    c_7920   m   1956
8619    c_7403   f   1970
8620    c_5119   m   1974
8621    c_5643   f   1968
8622      c_84   f   1982

[8623 rows x 3 columns]


In [17]:
# check to clean for bad data
customers.describe()

,birth
count,8623.000000
mean,1978.280877
std,16.919535
min,1929.000000
25%,1966.000000
50%,1979.000000
75%,1992.000000
max,2004.000000


In [18]:
# Check missing by variable
customers.isnull().sum()

client_id    0
sex          0
birth        0
dtype: int64

In [19]:
df = d1.merge(customers, left_on = 'client_id', right_on ='client_id')
print(df)

       id_prod                        date session_id client_id  price  categ  \
0       0_1483  2021-04-10 18:37:28.723910    s_18746    c_4450   4.99      0   
1       0_1085  2021-09-29 11:14:59.793823    s_97382    c_4450   3.99      0   
2       0_1453  2021-08-27 19:50:46.796939    s_81509    c_4450   7.99      0   
3       0_1405  2021-08-27 20:07:25.878440    s_81509    c_4450   4.99      0   
4       0_1392  2021-12-28 11:45:04.072281   s_141302    c_4450   6.30      0   
...        ...                         ...        ...       ...    ...    ...   
336708   1_607  2021-09-25 07:26:00.224331    s_95185    c_4786  26.99      1   
336709   1_673  2021-06-01 00:49:49.781631    s_42350    c_2793  12.99      1   
336710  0_2075  2021-10-09 09:03:48.268536   s_102200    c_2793   8.99      0   
336711  0_1692  2021-09-15 19:42:08.596375    s_90430    c_4478  13.36      0   
336712   0_142  2021-09-25 18:07:25.880052    s_95415    c_1232  19.85      0   

       sex  birth  
0      

In [20]:
# check to clean for bad data
df.describe()

,price,categ,birth
count,336713.000000,336713.000000,336713.000000
mean,17.215189,0.430156,1977.823568
std,17.855445,0.591082,13.524433
min,0.620000,0.000000,1929.000000
25%,8.610000,0.000000,1971.000000
50%,13.900000,0.000000,1980.000000
75%,18.990000,1.000000,1987.000000
max,300.000000,2.000000,2004.000000


In [21]:
# Check missing by variable
df.isnull().sum()

id_prod       0
date          0
session_id    0
client_id     0
price         0
categ         0
sex           0
birth         0
dtype: int64